# How to use st_hsdatalog HSD_link package - \[Communication Features\]
---
<br>
<span style="color:cyan; font-weight:bold">[!!!] Connect an HSDatalog v2 compatible device, then import the HSD_link module. [!!!]</span>
<p>Using this module with a connected device you will be able to:</p>
<ul>
<li>retrieve configuration details (both capabilities and current setting)</li>
<li>update device sensors parameters</li>
<li>control an acquisition saving sensor data transfered</li>
<li>annotate real-time acquisitions</li>
</ul>

In [ ]:
from st_hsdatalog.HSD_link.HSDLink import HSDLink

Initialize the HSD_link module with a specific device command set. 'st_hsd' string is used to create the right CommandManager internally.

In [ ]:
hsd_link = HSDLink()
hsd_link_instance = hsd_link.create_hsd_link(dev_com_type='st_hsd', acquisition_folder='./test_folder')

"st_hsd" USB communication is based on a DLL which guarantees stable and robust communication at very high speeds.
<p>To get DLL version:</p>

In [ ]:
version = hsd_link.get_version(hsd_link_instance)
print(f"HSDLink Version: {version}")

<p>This communication framework allows you to interact with a connected device using commands that could be grouped by several different types:</p>
<h2>GET commands</h2>
<hr>
<p>You can obtain easily the list of all connected Devices</p>

In [ ]:
devices = hsd_link.get_devices(hsd_link_instance)
print(f"Connected Devices: {devices}")

Or select a specific one

In [ ]:
device_id = 0

device = hsd_link.get_device(hsd_link_instance, device_id)
print(f"Device: {device}")

device_alias = hsd_link.get_device_alias(hsd_link_instance, device_id)
print(f"Device Alias: {device_alias}")

Create a new HSDatalog to store the device model obtained from the physical board.
<p><i>* All operations in this notebook can also be performed without using this <b style="color:cyan">[HSDatalog]</b> object, which however makes it easier to perform some tasks</i></p>

In [ ]:
from st_hsdatalog.HSD.HSDatalog import HSDatalog
hsd_factory = HSDatalog()
hsd= hsd_factory.create_hsd(device_config = device)

HSDatalog.set_device(hsd, device)

A list of high-level device information could be retrieved from the device model obtained
<p style="font-style: italic">the same thing can be done also using the <b style="color:cyan">[HSDatalog]</b> object </p>

In [ ]:
device_info = hsd_link.get_device_info(hsd_link_instance, device_id)
HSDatalog.present_device_info(hsd, device_info)
device_info = HSDatalog.get_device_info(hsd)
HSDatalog.present_device_info(hsd, device_info)

The list of sensors configuration can be obtained directly from the connected device and filtered by two parameters: the activation status and the sensor type.
<p><li> <u>Only active sensors</u> (<b style="color:cyan">[HSDatalog]</b> utility function is used to present the sensor list): </p>

In [ ]:
sensor_list = hsd_link.get_sensor_list(hsd_link_instance, device_id, only_active=True)
HSDatalog.present_sensor_list(hsd, sensor_list)

<p><li><u>All sensors</u>, even those not active (<b style="color:cyan">[HSDatalog]</b> utility function is used to present the sensor list): </p>

In [ ]:
sensor_list = hsd_link.get_sensor_list(hsd_link_instance, device_id, only_active=False)
HSDatalog.present_sensor_list(hsd, sensor_list)

<p><li><u>Active sensors filtered by a specific type</u> (e.g. MIC) (<b style="color:cyan">[HSDatalog]</b> utility function is used to present the sensor list): </p>

In [ ]:
sensor_list = hsd_link.get_sensor_list(hsd_link_instance, device_id, only_active=True, type_filter="MIC")
HSDatalog.present_sensor_list(hsd, sensor_list)

<p>Sensors configuration list can also be extracted from the previously retrieved Device object using the <b style="color:cyan">[HSDatalog]</b> object</p>

In [ ]:
sensor_list = HSDatalog.get_sensor_list(hsd, type_filter="ACC")
HSDatalog.present_sensor_list(hsd, sensor_list)


<p>It is also possible to directly select a specific sensor from the device model using the <b style="color:cyan">[HSDatalog]</b> object.

In [ ]:
sensor_name = "ism330dhcx_acc"
try:
    sensor = HSDatalog.get_sensor(hsd, sensor_name)
    HSDatalog.present_sensor(hsd, sensor)
except Exception as e:
    print(f"Choosen sensor [{sensor_name}] is missing in your connected device. Please choose another sensor.")

And to extract specific sensor parameters specifying only the sensor name.

In [ ]:
sensor_name = "iis2dlpc_acc"
sensor_enabled = hsd_link.get_sensor_enabled(hsd_link_instance, device_id, sensor_name)
sensor_odr = hsd_link.get_sensor_odr(hsd_link_instance, device_id, sensor_name)
sensor_fs = hsd_link.get_sensor_fs(hsd_link_instance, device_id, sensor_name)
sensor_spts = hsd_link.get_sensor_spts(hsd_link_instance, device_id, sensor_name)
print(f"Enabled: {sensor_enabled}")
print(f"ODR: {sensor_odr}")
print(f"FS: {sensor_fs}")
print(f"SamplePerTimestamp: {sensor_spts}")

<p>In Datalog2 some device Properties could be defined as Enumerative. Their values are defined in the related DTDL Property.</p><p>A way to get the numerical value for the related Property is described in the following code cell.</p></span>

In [ ]:
sensor = HSDatalog.get_sensor(hsd, sensor_name)
print(f"Sensor: {sensor}")
print(f"ODR: {HSDatalog.get_sensor_odr(hsd, sensor)}")
print(f"FS: {HSDatalog.get_sensor_fs(hsd, sensor)}")

<p>In addition you can get other information about available Tag classes in FW:</p>
<li><u>SW Tag</u> classes

In [ ]:
sw_tag_classes = hsd_link.get_sw_tag_classes(hsd_link_instance, device_id)
HSDatalog.present_sw_tag_classes(hsd, sw_tag_classes)
print(hsd_link.get_sw_tag_class_label_by_id(hsd_link_instance, device_id, tag_class_id=0))
print(hsd_link.get_sw_tag_class_enabled_by_id(hsd_link_instance, device_id, tag_class_id=0))
print(hsd_link.get_sw_tag_class_status_by_id(hsd_link_instance, device_id, tag_class_id=0))
print(hsd_link.get_sw_tag_class_label(hsd_link_instance, device_id, tag_class_name="sw_tag0"))
print(hsd_link.get_sw_tag_class_enabled(hsd_link_instance, device_id, tag_class_name="sw_tag0"))
print(hsd_link.get_sw_tag_class_status(hsd_link_instance, device_id, tag_class_name="sw_tag0"))

<p><li><u>HW Tag</u> classes</p>

In [ ]:
hw_tag_classes = hsd_link.get_hw_tag_classes(hsd_link_instance, device_id)
if hw_tag_classes is not None:
    HSDatalog.present_hw_tag_classes(hsd, hw_tag_classes)
else:
    print("No HW Tag classes available")

<li><u>Maximum number of annotations</u> supported per single acquisition</li>

In [ ]:
max_tag_per_acq = hsd_link.get_max_tags_per_acq(hsd_link_instance, device_id)
print(f"Max tags per acquisition: {max_tag_per_acq}")

<h2>SET Commands
<hr>

SET single sensor parameter:

In [ ]:
sensor_name = "ism330dhcx_acc"
print("--> SET enable Property = True")
hsd_link.set_sensor_enable(hsd_link_instance, device_id, True, sensor_name=sensor_name)
print(f"Enable: {hsd_link.get_sensor_enabled(hsd_link_instance, device_id, sensor_name=sensor_name)}")
print("--> SET odr Property = 1666.0 Hz (DTDL enum=7)")
hsd_link.set_sensor_odr(hsd_link_instance, device_id, 7, sensor_name=sensor_name)
print(f"ODR: {hsd_link.get_sensor_odr(hsd_link_instance, device_id, sensor_name=sensor_name)}")
print("--> SET fs Property = 8 g (DTDL enum=2)")
hsd_link.set_sensor_fs(hsd_link_instance, device_id, 2, sensor_name=sensor_name)
print(f"FS: {hsd_link.get_sensor_fs(hsd_link_instance, device_id, sensor_name=sensor_name)}")
print("--> SET SamplePerTimestamp = 500")
hsd_link.set_sensor_spts(hsd_link_instance, device_id, 500, sensor_name=sensor_name)
print(f"SamplePerTimestamp: {hsd_link.get_sensor_spts(hsd_link_instance, device_id, sensor_name=sensor_name)}")

Upload an ucf file to configure the MLC sensor on the board (also if it isn't enabled)

In [ ]:
ucf_file_path = "path/to/mlc.ucf"
hsd_link.upload_mlc_ucf_file(hsd_link_instance, device_id, ucf_file_path)

Device configuration update from JSON file

In [ ]:
json_file_path = "path/to/device_config.json"
try:
    hsd_link.update_device(hsd_link_instance, device_id, json_file_path)
except Exception as e:
    print(f"Wrong device_config.json: {e}")
sensor_list = hsd_link.get_sensor_list(hsd_link_instance, device_id, only_active=False)
HSDatalog.present_sensor_list(hsd, sensor_list)

Set next acquisition name and description

In [ ]:
hsd_link.set_acquisition_info(hsd_link_instance, device_id, "acq_name_00", "acq_notes_00")
print(hsd_link.get_acquisition_info(hsd_link_instance, device_id))

Configure SW Tags labels

In [ ]:
sw_tag_classes = hsd_link.get_sw_tag_classes(hsd_link_instance, device_id)
print(sw_tag_classes)

hsd_link.set_sw_tag_class_enabled_by_id(hsd_link_instance, device_id, 3, False)
hsd_link.set_sw_tag_class_label_by_id(hsd_link_instance, device_id, 3, "Walking")
hsd_link.set_sw_tag_class_enabled(hsd_link_instance, device_id, "sw_tag4", False)
hsd_link.set_sw_tag_class_label(hsd_link_instance, device_id, "sw_tag4", "Walking")

sw_tag_classes = hsd_link.get_sw_tag_classes(hsd_link_instance, device_id)
print(sw_tag_classes)

<h2>LOG CONTROL Commands </h2>
<hr>

Set RTC Time. Send to the board the current datetime formatted as follow:
<ul><b>"%Y%m%d_%H_%M_%S"</b></ul>

In [ ]:
hsd_link.set_RTC(hsd_link_instance, device_id)

Once the configuration phase is concluded, it is possible to start the acquisition process.
<p>With an acquisition in progress, you can get sensor data with for each active sensors.</p>
<p><i>* In the following three code cells we start an acquisition retrieving data from all active sensors and save them in the acquisition folder. A dedicated thread will be created for each active sensor</i></p>

In [ ]:
import os
import time
from threading import Event
from st_hsdatalog.HSD_link.HSDLink import SensorAcquisitionThread

threads_stop_flags = []
sensor_data_files = []

sensor_list = hsd_link.get_sensor_list(hsd_link_instance, device_id, only_active=True)
HSDatalog.present_sensor_list(hsd, sensor_list)

hsd_link.start_log(hsd_link_instance, device_id)

for sensor_name in sensor_list:
    is_enabled = hsd_link.get_sensor_enabled(hsd_link_instance, device_id, sensor_name)
    if is_enabled:
        sensor_data_file_path = os.path.join(hsd_link.get_acquisition_folder(hsd_link_instance), f"{sensor_name}.dat")
        sensor_data_file = open(sensor_data_file_path, "wb+")
        sensor_data_files.append(sensor_data_file)
        stopFlag = Event()
        threads_stop_flags.append(stopFlag)
        thread = SensorAcquisitionThread(stopFlag, hsd_link_instance, sensor_data_file, device_id, sensor_name)
        thread.start()
    else:
        print(f"[WARNING]: Sensor {sensor_name} is not enabled.")

time.sleep(3)

Tag an acquisition in progress. (wait for 3 seconds, then 3 seconds of SW Tag Class 0 ->(label defined before as "Walking"), followed by 3 seconds of SW Tag Class 1.

In [ ]:
time.sleep(3)

tag_id = 0
hsd_link.set_sw_tag_on(hsd_link_instance, device_id, tag_id)
time.sleep(3)
hsd_link.set_sw_tag_off(hsd_link_instance, device_id, tag_id)

hsd_link.set_sw_tag_on(hsd_link_instance, device_id, 1)
time.sleep(3)
hsd_link.set_sw_tag_off(hsd_link_instance, device_id, 1)

time.sleep(3)

<p>To stop the process and save all necessary acquisition files in the right folder, you have to launch the following commands.</p>
<p>In addition to sensor data files(.dat), the following JSON files will also be saved in the acquisition folder:</p>
<li>Device configuration file <b>[device_config.json]</b> containing the current device status snapshot (Properties values)</li>
<li>Acquisition info file <b>[acquisition_info.json]</b> containing a list of all annotations taken during the acquisition process (Acquisition Info Component Status)</li>
<br>
<i>* Each of these files contains an identification string to link them together and with a specific acquisition</i>

In [ ]:
for sf in threads_stop_flags:
    sf.set()
for f in sensor_data_files:
    f.close()

hsd_link.stop_log(hsd_link_instance, device_id)
hsd_link.save_json_device_file(hsd_link_instance, device_id)
hsd_link.save_json_acq_info_file(hsd_link_instance, device_id)